This is the main notebook of the report, it'll contain all of our findings and developments.

In [5]:
import DataReader
import Word2VecAverage

x_test_set, x_train_set, y_test_set, y_train_set = DataReader.split_sheet_into_test_training_per_word()
w2v = Word2VecAverage.Word2VecAverage()
w2v.train_word2vec()

x_test_features = w2v.create_feature_set(x_test_set)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
clf = LogisticRegression()
clf.fit(x_test_features, y_test_set)
x_train_features = w2v.create_feature_set(x_train_set)
test_pred_clf = clf.predict(x_test_features)
metrics.accuracy_score(test_pred_clf, y_test_set)

C:\Users\lolco\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\lolco\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6892121962544497

# Ngram

In [7]:
import Ngram
from DataReader import split_sheet_into_test_training_per_word
x_test, x_train, y_test, y_train = split_sheet_into_test_training_per_word()
n=2

One model:

In [8]:
x_clean = Ngram.process_1(x_train, y_train, n)
model = Ngram.get_model( x_clean,n)
print('correct, error, doesn\'t know (in%)')
print(Ngram.compute_score(x_test, y_test, model, n))

correct, error, doesn't know (in%)
(0.6562451632874168, 0.04550379198266522, 0.298251044729918)


Multiple models (one per definition):

In [9]:
x1,x2,x3,x4 = Ngram.process_4(x_train, y_train, n)
models = []
xs = [x1,x2,x3,x4]
for x in xs:
    models.append(Ngram.get_model(x, n))
print('correct, error, doesn\'t know (in%)')
print(Ngram.compute_score(x_test, y_test, models, n))

correct, error, doesn't know (in%)
(0.6489707475622969, 0.05277820770778517, 0.298251044729918)


# BoW approach

In [10]:
import Bow
mod = Bow.Bow(x_train, y_train)
print("correct, wrong, noidea")
print(mod.get_score(x_test,y_test))

correct, wrong, noidea
(0.8277356446370531, 0.09642470205850487, 0.07583965330444203)


# Baseline

In [8]:
from MostCommon import MostCommon
from sklearn import metrics
import numpy as np
mc = MostCommon()
counts = mc.createCounts()
predicted = mc.classify(mc.x_test, counts)
testl = np.ravel(mc.y_test)
print(metrics.classification_report(testl, predicted, zero_division=1))

              precision    recall  f1-score   support

           -       0.00      0.00      0.00        74
           1       0.94      0.11      0.19      4364
           2       0.33      0.92      0.48      1830
           3       0.23      0.99      0.37       163
           4       0.28      1.00      0.44        30

    accuracy                           0.36      6461
   macro avg       0.35      0.60      0.30      6461
weighted avg       0.73      0.36      0.28      6461



# Semantic dependencies

In [7]:
from SemanticDep import SemanticDep
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
sd = SemanticDep()
f = sd.createFeatures(sd.x_train)
l = np.ravel(sd.y_train)
testf = sd.createFeatures(sd.x_test)
testl = np.ravel(sd.y_test)

# train the model on the features
clf = MultinomialNB()
clf.fit(f, l)

# predict the labels for the test data
predicted = clf.predict(testf)
# output your results
print(metrics.classification_report(testl, predicted))

              precision    recall  f1-score   support

           -       0.15      0.77      0.25        86
           1       0.89      0.71      0.79      4375
           2       0.64      0.69      0.66      1815
           3       0.29      0.75      0.42       153
           4       0.21      0.94      0.35        32

    accuracy                           0.71      6461
   macro avg       0.43      0.77      0.49      6461
weighted avg       0.79      0.71      0.74      6461

